In [ ]:
import numpy as np
import pandas as pd
import torch
import re
import torch
from tqdm import notebook
from itertools import combinations
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist, cosine
from scipy.stats.stats import pearsonr

from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 24.0 MB/s 
     |████████████████████████████████| 7.6 MB 31.1 MB/s 
     |████████████████████████████████| 163 kB 69.2 MB/s 


In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#BERT = "DeepPavlov/rubert-base-cased"
BERT = "cointegrated/rubert-tiny2"

In [ ]:
data = pd.read_csv('заказы_описание.csv', delimiter=",", encoding='utf-8')
data

,orderId,specialityId,description,latitude,longitude
0,e881f3f7-0bc6-4763-ba2c-87cf8a21d5f6,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,55.818270,37.965373
1,388434f8-41a7-4e2c-b382-e7e524ea4392,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,52.227534,104.304025
2,462aa394-9b06-4018-9993-623ef196ee81,4f1a1b97-21c8-4f64-badf-ab4760eebaed,"Оказание услуг по выкладке товара в зале, заме...",55.827687,37.521264
3,1795415b-ab57-4409-9e1f-12070537dd58,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,55.612182,37.607026
4,d45b53e5-2c19-4931-a606-0b9bb435a444,4f1a1b97-21c8-4f64-badf-ab4760eebaed,"Уборка ТЗ, помывка оборудования, переоценка, п...",53.220574,50.262926
...,...,...,...,...,...
90884,7e803c1c-f0f8-4f58-a4e6-00cd34474072,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,55.794978,37.936555
90885,18f07585-009c-46f7-9419-a31820cc94bc,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,47.708491,40.215563
90886,56659a25-eeac-4e9b-b32f-ded08bed9666,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Своевременное пополнение мест продажи в торгов...,44.100971,39.092371
90887,d851fa71-2271-49ef-8478-f917ed5cf865,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,44.868677,38.151315


In [ ]:
data1 = pd.read_csv('исполнители_описание.csv', sep=',')
data1

,contractorId,latitude,longitude,specialityId,description,mainSpecialityId
0,a3cd6a6e-a5bb-49b2-b12a-364097e2e48d,54.988526,73.383088,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,4f1a1b97-21c8-4f64-badf-ab4760eebaed
1,00b18f5e-f67c-4999-bfb8-e71fb1db9b9d,54.937820,73.381112,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,4f1a1b97-21c8-4f64-badf-ab4760eebaed
2,b0a23d2c-d376-4c37-a25c-5f4833335493,55.719602,37.556038,c05549af-088a-43de-bd10-5f57fc3e389e,Сборка корпусов установок очистки и обеззаражи...,db7a9156-36ae-408a-a5df-955ab8738b57
3,258245e0-ceeb-4ac1-b26f-180f075dfd2b,55.719602,37.556038,c05549af-088a-43de-bd10-5f57fc3e389e,Сборка корпусов установок очистки и обеззаражи...,c05549af-088a-43de-bd10-5f57fc3e389e
4,55cd1b2a-d371-4102-a1fd-3abe1371d342,55.719602,37.556038,73718e83-7de1-464f-bc25-537243d2759e,доставка персонала на личном автомобиле,623c4e3e-5971-43f7-895a-c9e68abf9ede
...,...,...,...,...,...,...
107434,f2b6e747-b618-4bf2-b8f1-e89a8bfb35fe,59.939095,30.315868,d55e4589-beac-4964-bd69-d7efff11290c,Услуги по выкладке товара в торговом зале,d55e4589-beac-4964-bd69-d7efff11290c
107435,eae4bcd3-bb4f-4d3f-96ba-983222400b01,53.724593,91.421825,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,4f1a1b97-21c8-4f64-badf-ab4760eebaed
107436,2bec24b9-23d8-4308-9551-74c43c3f45c6,64.524220,40.601758,623c4e3e-5971-43f7-895a-c9e68abf9ede,Выкладка товара в магазине MODIS,d55e4589-beac-4964-bd69-d7efff11290c
107437,c0ea2f25-3c68-4d5c-bcf6-02af5d0ee3d6,60.059086,30.335020,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,d55e4589-beac-4964-bd69-d7efff11290c


In [ ]:
data1=data1.drop_duplicates(subset=['contractorId','description'])

In [ ]:
data1=data1[0:5000]
data1

,contractorId,latitude,longitude,specialityId,description,mainSpecialityId
0,a3cd6a6e-a5bb-49b2-b12a-364097e2e48d,54.988526,73.383088,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,4f1a1b97-21c8-4f64-badf-ab4760eebaed
1,00b18f5e-f67c-4999-bfb8-e71fb1db9b9d,54.937820,73.381112,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,4f1a1b97-21c8-4f64-badf-ab4760eebaed
2,b0a23d2c-d376-4c37-a25c-5f4833335493,55.719602,37.556038,c05549af-088a-43de-bd10-5f57fc3e389e,Сборка корпусов установок очистки и обеззаражи...,db7a9156-36ae-408a-a5df-955ab8738b57
3,258245e0-ceeb-4ac1-b26f-180f075dfd2b,55.719602,37.556038,c05549af-088a-43de-bd10-5f57fc3e389e,Сборка корпусов установок очистки и обеззаражи...,c05549af-088a-43de-bd10-5f57fc3e389e
4,55cd1b2a-d371-4102-a1fd-3abe1371d342,55.719602,37.556038,73718e83-7de1-464f-bc25-537243d2759e,доставка персонала на личном автомобиле,623c4e3e-5971-43f7-895a-c9e68abf9ede
...,...,...,...,...,...,...
6046,9aa7b9b8-feba-497b-8612-4a74fc734dd0,55.785979,37.660521,d951f4ef-ad9a-47ed-bc5d-eba397d4b7ec,21-053407-01-01 Исследование переключения марк...,d951f4ef-ad9a-47ed-bc5d-eba397d4b7ec
6047,f373db72-83f0-4a48-be51-39a7df7e7858,55.055649,82.911874,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,4f1a1b97-21c8-4f64-badf-ab4760eebaed
6049,d0bb34c6-021c-48eb-89ee-83eee87ef614,55.963889,37.921329,c4cfc9e9-0063-4bd7-9202-f9a54ee141f4,Задачи:\nВыкладка основного товара (количество...,c4cfc9e9-0063-4bd7-9202-f9a54ee141f4
6051,8b7e168c-aff2-4579-94c2-4d08f3422c43,44.552348,33.529631,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,d55e4589-beac-4964-bd69-d7efff11290c


In [ ]:
# загружаем два алгоритма
# токенайзер - инструмент перевода слов в токены - числовые значения слов в словаре
# и модель превращения вектора с токенами в эмбеддинги

tokenizer = AutoTokenizer.from_pretrained(BERT)
model = AutoModel.from_pretrained(BERT)

Downloading:   0%|          | 0.00/401 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#расчет векторов для исполнителей
corpus = data1['description'].str.lower().values

# удаляем из текста ненужные знаки препинания
corpus_new = []
for text in corpus:
    text = re.sub(r'\xa0', ' ', text)
    text = re.sub(r'«»,:', '', text)
    corpus_new.append(text)

# переводим предложения в векторы
# каждому слову соответсвует определеное число в словаре
tokenized = []
for i in corpus_new:
    tokenized.append(tokenizer.encode(i, add_special_tokens=True))

# рассчитываем максимальную длину маски внимания
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)

# для каждого токенизированного предложения создаем маску внимания и приводим векторы к единой длине
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized])
attention_mask = np.where(padded != 0, 1, 0)

# сколько предложений будет обработано за одну итерацию
batch_size = 1

# c помощью модели BERT переводим токенизированные предложения в эмбеддинги
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size * i : batch_size * (i + 1)])
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size * i : batch_size * (i + 1)])

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/39077 [00:00<?, ?it/s]

In [ ]:
corpus_new = []
for text in corpus:
    text = re.sub(r'\xa0', ' ', text)
    corpus_new.append(text)

In [ ]:
corpus_new[2]

'проверка соответствия и наличия ценников, пополнение товарного запаса и ассортимента в торговом зале, поддержание порядка на складе и в торговом зале.'

In [ ]:
tokenized[2]

[101,
 30656,
 47212,
 851,
 32623,
 11992,
 3636,
 128,
 71208,
 71968,
 41567,
 851,
 70407,
 626,
 845,
 51754,
 22121,
 128,
 50796,
 15197,
 1469,
 57207,
 851,
 845,
 51754,
 22121,
 132,
 102]

In [ ]:
data2=data[0:1]
data2

,orderId,specialityId,description,latitude,longitude
0,e881f3f7-0bc6-4763-ba2c-87cf8a21d5f6,4f1a1b97-21c8-4f64-badf-ab4760eebaed,Работа с товаром в торговом зале обувного мага...,55.81827,37.965373


In [ ]:
#для заказа
#расчет векторов для исполнителей
corpus1 = data2['description'].str.lower().values

# удаляем из текста ненужные знаки препинания
corpus_new1 = []
for text in corpus1:
    text = re.sub(r'\xa0', ' ', text)
    text = re.sub(r'«»,:', '', text)
    corpus_new1.append(text)

# переводим предложения в векторы
# каждому слову соответсвует определеное число в словаре
tokenized1 = []
for i in corpus_new1:
    tokenized1.append(tokenizer.encode(i, add_special_tokens=True))

# рассчитываем максимальную длину маски внимания
max_len1 = 0
for i in tokenized1:
    if len(i) > max_len1:
        max_len1 = len(i)

# для каждого токенизированного предложения создаем маску внимания и приводим векторы к единой длине
padded = np.array([i + [0]*(max_len1 - len(i)) for i in tokenized1])
attention_mask = np.where(padded != 0, 1, 0)

# сколько предложений будет обработано за одну итерацию
batch_size = 1

# c помощью модели BERT переводим токенизированные предложения в эмбеддинги
embeddings1 = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size * i : batch_size * (i + 1)])
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size * i : batch_size * (i + 1)])

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings1.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
corpus_new1[0]

'работа с товаром в торговом зале обувного магазина. оплата: обработка 1 короба - 100 руб. смена ценников 200 ед. товара - 100 руб.'

In [ ]:
# выбираем пример, относительно которого будем подбирать похожие новости
example = embeddings1[0]

# два списка, в которые будем отбирать претендетов на показ
idx = []
cos_sim = []

# пробегаемся списком по всем эмбеддингам и выбираем те, где косинусное расстояние от образца менее 15%.
# или где косинусная схожесть больше или равна 85%
for i, j in enumerate(embeddings):
    if cosine_similarity(example, j) >= 0.85:
        idx.append(i)
        cos_sim.append(cosine_similarity(example, j)[0][0])

# собираем все найденные заголовки в новый датафрейм, добавляем конисусное расстояние и выводим топ-11 подобных заголовков
# заголовок со схожестью 1 - образец
cos_sim = pd.Series(cos_sim, name='cos_sim', index=idx)
res = data1.query('index in @idx').copy()
res = res.join(cos_sim)

res = res.sort_values(by='cos_sim', ascending=False)[:11]

res

,contractorId,orderId,description,cos_sim
22,b3ee1e88-77cf-4cba-abeb-797597e39d91,0003e0a6-8147-4a12-a192-ac412080eb15,Работа с товаром в торговом зале обувного мага...,1.000000
10,299dc1c8-3d8d-43b6-8f0a-689e43595f8d,00028efd-710b-4074-9792-8d1f6c600175,Работа с товаром в торговом зале обувного мага...,1.000000
72,70bb6c18-931f-4a75-ba3b-eda5b17deb19,000767a8-ec17-4570-a158-4a2f271bed96,Работа с товаром в торговом зале обувного мага...,1.000000
43,ad986935-353d-49ea-9fc0-e0168a73470d,0005489a-a7cc-4624-8a29-038e8eb8de61,Работа с товаром в торговом зале обувного мага...,0.988145
70,b199f024-af63-4ec9-80bb-6e692b521501,00074fcb-e684-4ed9-8f9e-4e4c7b56889b,"Посещение торговой точки, проведение аудита вы...",0.945475
56,15d0698d-de8c-477e-a2ed-67465319f80a,000660be-b0a3-4d0a-b59e-153abb8a5a85,"Посещение торговой точки, проведение аудита вы...",0.945433
9,df2c2a9c-5a8d-4e18-8815-8894fbb54c28,00028e94-e7e8-4c71-bff3-ab0e0383a1b7,Оказание услуг охранника/контролёра в обувном ...,0.944404
6,b8fa1d38-cdb9-4854-82d2-b5ed318d1e82,0002762f-874a-4c32-ad2e-4b3d7241eb62,"Проверка соответствия и наличия ценников, попо...",0.923198
5,941966b0-e78c-4a62-84a9-e4589dbeec19,0002762f-874a-4c32-ad2e-4b3d7241eb62,"Проверка соответствия и наличия ценников, попо...",0.923198
4,5a739fbe-6bc5-4084-a1c8-095ae0f7a380,0002762f-874a-4c32-ad2e-4b3d7241eb62,"Проверка соответствия и наличия ценников, попо...",0.923198
